# Lecture 6: Cleaning It All Up

(Summer 2023)


In [ ]:
# Bring in the packages we have used before.

import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy.linalg as la

In [ ]:
# What version of pandas are we running ... ?
pd.__version__

## Summary of Friday ...

* Intro to the basic pandas data structures: `index`, `series`, and `dataframe`
* How to construct them ... `pd.Series`, `pd.DataFrame`
* How to access the insides of series and dataframes ... `df.index`, `df.values`, `df[3]`, `df[3:7]`, `df.head()`, `df.tail()`, `df.describe()`
* Comparison of numpy arrays and pandas dataframes (e.g., implicit indices vs. explicit indices)
* Sorting: `df.sort_index(axis={0 or 1}, ascending={True or False})`, `df.sort_values(by='{a column label}')`
* Converting between dataframes and numpy: `d = df.to_numpy()`, `df = pd.DataFrame(d)`
* Reading a csv file and creating a dataframe: `pandas.read_csv()`
* We opened two datasets `wheatyields` and `cities_and_towns` to illustrate some pandas methods
* Finallly we opened two datasets `WheatYields--Wrangled` and `Wrangled-cities-and-towns-of-the-united-states` to illustrate merging of dataframes and plotting from dataframes.

<a href="https://pandas.pydata.org/docs/user_guide/10min.html" target="_blank">10 Minutes to Pandas</a>

## In this lecture we will pick up from where we left off and try to explore a few more tools, mostly looking at real data sets ...

## Bring in Cities and Towns of the US Data Set ...

We will bring in the original and demonstrate wrangling it using pandas tools rather than the low level methods previously explored ...

In [ ]:
CitiesAndTowns = pd.read_csv('Data/cities-and-towns-of-the-united-states.csv')
CitiesAndTowns

### Certainly not happy with this. Looking at the header line we see that the issue is likely with the separator, i.e., they are using `;` rather than `,`

So let's go to the `pandas.read_csv` documentation ...

<a href="https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html" target="_blank">pandas.read_csv</a>

The parameter `sep` needs to be changed from its default value ...

In [ ]:
CitiesAndTowns = pd.read_csv('Data/cities-and-towns-of-the-united-states.csv', sep=';')

In [ ]:
CitiesAndTowns

### That's better!

In [ ]:
CitiesAndTowns.columns

## Decoding the data ...

Notice the complexity of the data and the seming redundancy. There are 18 header items:

0. Geo Point: is a lat, lon pair, e.g., 44.2998540717,-120.834480347
1. Geo Shape: is a Json-like object, e.g., "{""type"": ""Point"", ""coordinates"": [-120.83448034701922, 44.299854071684514]}"
2. GNIS ID: is an integer, e.g., 1154317
3. ANSICODE: is an integer, e.g., 2411494
4. FEATURE: is a text descriptor, e.g., Civil
5. FEATURE2: is a text descriptor, e.g., County Seat
6. NAME: seems to be the name of the Feature, e.g., Prineville (the name of a city apparently)
7. POP 2010: probably the population of the entity in 2010, e.g., 9253
8. COUNTY: name, e.g., Crook
9. COUNTY FIPS: a 3 digit integer uniquely identifying the county in the state, e.g., 013
10. STATE: name abbreviation, e.g., OR
11. STATE FIPS: a 2 digit code identifying the state, e.g., 41
12. LATITUDE: similar number, e.g., 44.2998485, but with less precision
13. LONGITUDE: similar number, e.g., -120.8344664, but with less precision
14. PopPlLat: seems are repetition of the above number, e.g., 44.2998485
15. PopPlLong: seems a repetition of the above number, e.g., -120.8344664
16. ELEV IN M: 878
17. ELEV IN FT: 2881

To better understand some of these, it would be helpful to see the range of values that are present in the data. Of course, this can be easily done with excel.

## In the next cell I pull out the information of interest ...

Essentially I need to create a "location" for each state-county pair. Then I might be able to create maps colored by data from the USDA web page. Therefore, the items of most interest are ...

`FEATURE`
`FEATURE2`
`POP 2010`
`COUNTY`
`COUNTY FIPS`
`STATE`
`STATE FIPS`
`LATITUDE`
`LONGITUDE`
`ELEV IN FT`


In [ ]:
# Lets create a new CitiesAndTowns with only the info desired ...

CitiesAndTowns = CitiesAndTowns[['FEATURE', 'FEATURE2', 'POP 2010', 'COUNTY', 'COUNTY FIPS', 'STATE', 'STATE FIPS', 'LATITUDE', 'LONGITUDE', 'ELEV IN FT']]
print(CitiesAndTowns)

### Lets look closer a FEATURE2 ...

In [ ]:
CitiesAndTowns['FEATURE2'].unique()

### Of most interest regarding counties are the labels

`County Seat` and `State Capital County Seat`

In [ ]:
Flag = (CitiesAndTowns['FEATURE2'] == 'County Seat') | (CitiesAndTowns['FEATURE2'] == 'State Capital County Seat')

In [ ]:
Flag

In [ ]:
CountySeats = CitiesAndTowns.loc[Flag]
CountySeats

### Indexers: `.loc` and `.iloc`, etc.

Some of the slicing and indexing operations we've learned in dealing with numpy arrays are a potential source of confusion when applied to pandas series and dataframes.

Above we used the `.loc` method so let's digress briefly to go over it ...

In [ ]:
# Keep this simple data for now ...

area_dict = {'Alaska': 665400, 'California': 163696, 'Texas': 268597, 'New York': 54556, 'Florida': 65758, 'Illinois': 57915, 'Indiana': 35826, 'Colorado': 104185}
pop_dict = {'Alaska': 732673, 'California': 39240000, 'Texas': 29530000, 'New York': 19840000, 'Florida': 21780000, 'Illinois': 12670000, 'Indiana': 6806000, 'Colorado': 5812000}
area = pd.Series(area_dict)
population = pd.Series(pop_dict)
states = pd.DataFrame({'population': population, 'area': area})
states


In [ ]:
area

In [ ]:
area[0]

In [ ]:
area['Alaska']

### Explicit index vs. Implicit index

In the example above the explicit index is `'Alaska'` and the corresponding implicit index is `0` ... Notice what happens in the following cases ...

In [ ]:
# Using .loc when the index is not one of its explicit values ...

area.loc[0]

In [ ]:
# Using .iloc when the index is not one of its implicit values (e.g., not an integer)

area.iloc['Alaska']

In [ ]:
# How does slicing work for implicit values ...

area[2:4]

In [ ]:
area.iloc[2:4]

In [ ]:
area['Texas':'Florida']

In [ ]:
area.loc['Texas':'Florida']

### Notice how slicing is done differently depending on if the index is explicit or implicit!

If the explicit indices are also integers, then there is a potential for confusion. To avoid this possible confusion, python provides `.loc` and `.iloc` to force one or the other interpretation. In the above example there would not be a problem. Following is an example where there is potential for confusion ...

In [ ]:
dexample = pd.Series(['a', 'b', 'c', 'd', 'e'], index = [1, 4, 6, 7, 99])
dexample

In [ ]:
# Explicit index when indexing

dexample[4]

In [ ]:
# But implicit index when slicing ...

dexample[1:4]

In [ ]:
# what if we try to use an implicit index ... ?
# This will cause an error.

dexample[0]

In [ ]:
# .loc forces the index to be explicit

dexample.loc[1:4]

In [ ]:
# .iloc forces the index to be implicit
dexample.iloc[1:4]

### Not using one or the other then falls to the python interpreter to decide ... Therefore, recommend to use one or the other

In [ ]:
# But note that in our original example, it would not have mattered ...

CountySeats2 = CitiesAndTowns[Flag]
CountySeats2

### Back to the `CountySeats` data and more wrangling aka clean-up ...

In [ ]:
# What are the values taken by FEATURE2 ... ? That is, just double checking
# that we got the desired result after applying the FLAG filter

CountySeats['FEATURE2'].unique()

In [ ]:
# Another data check ... Which reveals some impossible values apparently
# used to mark the existence of no data or some such ...

CountySeats['POP 2010'].min()

In [ ]:
# Create a flag index to filter out the impossible values ...

Flag = (CountySeats['POP 2010'] > 0)
Flag

In [ ]:
# Now filter using the recommended indexer ...

CountySeats = CountySeats.loc[Flag]

In [ ]:
# Did it work ... ?

CountySeats['POP 2010'].min()

## Implementing various plots as examples ...

In [ ]:
ax1 = CountySeats.plot.scatter(x='LONGITUDE',y='LATITUDE',s=1)
ax1.grid()

In [ ]:
CountySeats['ELEV IN FT']

In [ ]:
dv = CountySeats['ELEV IN FT']
dlat = CountySeats['LATITUDE']
dlon = CountySeats['LONGITUDE']
fig = plt.figure()
plt.style.use('classic')
plt.scatter(dlon, dlat, c=dv, cmap='cool')
plt.colorbar()
plt.title("Elevation of County Seats in ft.")
plt.xlabel("Longitude Degrees")
plt.ylabel("Latitude Degrees")
plt.grid()

In [ ]:
dv = CountySeats['POP 2010']
dlat = CountySeats['LATITUDE']
dlon = CountySeats['LONGITUDE']
fig = plt.figure()
plt.style.use('classic')
plt.scatter(dlon, dlat, c=dv, cmap='cool')
plt.colorbar()
plt.title("2010 Population of County Seats")
plt.xlabel("Longitude Degrees")
plt.ylabel("Latitude Degrees")
plt.grid()

In [ ]:
dv = CountySeats['POP 2010']
dlat = CountySeats['LATITUDE']
dlon = CountySeats['LONGITUDE']
fig = plt.figure()
plt.style.use('classic')
plt.scatter(dlon, dlat, s=dv/10000, alpha = 0.4) 
plt.title("2010 Population of County Seats")
plt.xlabel("Longitude Degrees")
plt.ylabel("Latitude Degrees")
plt.grid()

In [ ]:
fig = plt.figure()
plt.style.use('classic')
plt.scatter(dlon, dlat, s=dv/10000, alpha = 0.4) 
plt.title("2010 Population of County Seats")

plt.xlim(-130,-65)
plt.ylim(24,50)

plt.xlabel("Longitude Degrees")
plt.ylabel("Latitude Degrees")
plt.grid()

## Repeating the data clean up for WheatYields

In [ ]:
WheatYields = pd.read_csv('Data/WheatYields.csv')
WheatYields

In [ ]:
WheatYields.columns

In [ ]:
WheatYields = WheatYields[['Year', 'State', 'State ANSI', 'County', 'County ANSI', 'Value']]
print(WheatYields)

In [ ]:
WheatYields['County'].unique()

### Everything looks like a legitimate county name except for `'OTHER (COMBINED) COUNTIES'` Let's remove that ...

In [ ]:
Flag = (WheatYields['County'] != 'OTHER (COMBINED) COUNTIES')
Flag

In [ ]:
WheatYields = WheatYields.loc[Flag]
WheatYields

## What I'd like to do ...

### Merge these two tables to create a single table with the following columns ...

Year, State Ansi/Fips, County Ansi/Fips, Latitude, Longitude, Value (wheat yield in bu/acre)

This really amounts to deleting certain columns from the WheatYields dataframe and adding the columns for latitude and longitude


In [ ]:
CountySeats.index

In [ ]:
CountySeats.columns

In [ ]:
WheatYields.index

In [ ]:
WheatYields.columns

### Suppose you wanted to make a combined FIPs ....

Let the integer FIPs be defined by putting the state FIPs into the 1000s place and letting the county FIPS represent a number between 1 and 999 ...

In [ ]:
CountySeats.loc[:,'CombinedFIPS'] = 1000*CountySeats.loc[:,'STATE FIPS'] + CountySeats.loc[:,'COUNTY FIPS']
WheatYields.loc[:,'CombinedFIPS'] = 1000*WheatYields.loc[:,'State ANSI'] + WheatYields.loc[:,'County ANSI']

In [ ]:
CountySeats

In [ ]:
WheatYields

In [ ]:
CountySeats.values

In [ ]:
WheatYields.loc[WheatYields.Year == 2007, :]

### There is a lot of superfluous information ... Pare it down to simplify

In [ ]:
NewWheatYields = WheatYields.loc[:, ['Year', 'Value', 'CombinedFIPS']]
NewWheatYields

In [ ]:
CountySeats

In [ ]:
NewCountySeats = CountySeats.loc[(CountySeats.FEATURE2 == 'County Seat') | (CountySeats.FEATURE2 == 'State Capital County Seat'), ['LATITUDE', 'LONGITUDE', 'CombinedFIPS']]
NewCountySeats

In [ ]:
Blah = pd.merge(NewCountySeats, NewWheatYields)

In [ ]:
Blah.CombinedFIPS.unique()

In [ ]:
Blah

In [ ]:
Blah.loc[Blah.Year == 1999,:]

In [ ]:
dv = Blah.loc[Blah.Year == 1999,'Value']
dlat = Blah.loc[Blah.Year == 1999,'LATITUDE']
dlon = Blah.loc[Blah.Year == 1999,'LONGITUDE']
type(dv)

In [ ]:
fig = plt.figure()
plt.style.use('classic')
plt.scatter(dlon, dlat, c=dv, cmap='cool')
plt.colorbar()
plt.title("U.S. County Average Wheat (bu/acre)")
plt.xlabel("Longitude Degrees")
plt.ylabel("Latitude Degrees")
plt.grid()

In [ ]:
ddlat = CountySeats['LATITUDE']
ddlon = CountySeats['LONGITUDE']

In [ ]:
fig = plt.figure()
plt.style.use('classic')
plt.scatter(dlon, dlat, s=dv, alpha = 0.4)
plt.scatter(ddlon, ddlat, s=.1)

plt.xlim(-130,-65)
plt.ylim(24,50)

plt.title("U.S. County Average Wheat (bu/acre)")
plt.xlabel("Longitude Degrees")
plt.ylabel("Latitude Degrees")
plt.grid()

## Saving some code for later ...

In [ ]:
# Calc of Tippecanoe County ... nw, ne, se, sw
nw = np.array[[40.562935, -87.095078], [40.562042, -86.695375], [40.214476, -86.695723], [40.214925, -87.092605]]

In [ ]:
# Function computes the great circle (e.g., perfect sphere) model distance
# between two points (lon1, lat1) and (lon2, lat2). R is the radius of the
# sphere and the distance returned is in the same units as R.
#
# Note that lats and lons must be given in radians

def GCDist(lon1, lat1, lon2, lat2, R):
    temp = math.cos(lat1)*math.cos(lat2)*math.cos(lon1 - lon2) + math.sin(lat1)*math.sin(lat2)
    d = R*math.acos(temp)
    return d

In [ ]:
R = 3958.8
f = math.pi/180
D_nw_ne = GCDist(-87.095078*f,40.562935*f,-86.695375*f,40.562042*f,R)
D_nw_ne